# Defining a Fixed-Floating Interest Rate Swap

In [23]:
# This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; 
# without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  
# See the license for more details.

Simple introduction to the inputs required to define a Libor deposit and a FRA

In [24]:
from financepy.finutils.FinDate import FinDate
from financepy.finutils.FinDayCount import FinDayCountTypes
from financepy.finutils.FinFrequency import FinFrequencyTypes
from financepy.finutils.FinCalendar import *
from financepy.products.libor.FinLiborSwap import FinLiborSwap
from financepy.finutils.FinMath import ONE_MILLION

## Creating the Swap

We begin by setting the market conventions we wish to use.

In [25]:
swapCalendarType = FinCalendarTypes.TARGET
busDayAdjustType = FinDayAdjustTypes.FOLLOWING
dateGenRuleType = FinDateGenRuleTypes.BACKWARD

We then define the Fixed Leg

In [26]:
fixedCoupon = 0.05
fixedFreqType = FinFrequencyTypes.ANNUAL
fixedDayCountType = FinDayCountTypes.ACT_360

And then we define the Floating Leg

In [27]:
floatSpread = 0.0
floatFreqType = FinFrequencyTypes.SEMI_ANNUAL
floatDayCountType = FinDayCountTypes.ACT_360

We need to specify the size and direction of the swap.

In [28]:
payFixedFlag = False
notional = 10 * ONE_MILLION

And then we define when the swap begins and ends.

In [29]:
startDate = FinDate(20, 6, 2018)
maturityDate = FinDate(20, 9, 2025)

We can now create the swap object.

In [30]:
swap = FinLiborSwap(startDate,
                    maturityDate,
                    fixedCoupon,
                    fixedFreqType,
                    fixedDayCountType,
                    notional,
                    floatSpread,
                    floatFreqType,
                    floatDayCountType,
                    payFixedFlag,
                    swapCalendarType,
                    busDayAdjustType,
                    dateGenRuleType)

In [31]:
print(swap)

START DATE: WED 20 JUN 2018
MATURITY DATE: SAT 20 SEP 2025
NOTIONAL: 10000000
PAY FIXED LEG: False
FIXED COUPON: 0.05
FLOAT SPREAD: 0.0
FIXED FREQUENCY: FinFrequencyTypes.ANNUAL
FLOAT FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.ACT_360
FLOAT DAY COUNT: FinDayCountTypes.ACT_360
PAY FIXED FLAG: False
CALENDAR: FinCalendarTypes.TARGET
BUS DAY ADJUST: FinDayAdjustTypes.FOLLOWING
DATE GEN TYPE: FinDateGenRuleTypes.BACKWARD



In [32]:
swap.fixedDates()

[THU 20 SEP 2018,
 FRI 20 SEP 2019,
 MON 21 SEP 2020,
 MON 20 SEP 2021,
 TUE 20 SEP 2022,
 WED 20 SEP 2023,
 FRI 20 SEP 2024,
 MON 22 SEP 2025]

In [33]:
swap.floatDates()

[THU 20 SEP 2018,
 WED 20 MAR 2019,
 FRI 20 SEP 2019,
 FRI 20 MAR 2020,
 MON 21 SEP 2020,
 MON 22 MAR 2021,
 MON 20 SEP 2021,
 MON 21 MAR 2022,
 TUE 20 SEP 2022,
 MON 20 MAR 2023,
 WED 20 SEP 2023,
 WED 20 MAR 2024,
 FRI 20 SEP 2024,
 THU 20 MAR 2025,
 MON 22 SEP 2025]

## Valuation

In [34]:
valuationDate = FinDate(20,3,2020)

In [35]:
settlementDate = valuationDate.addWorkDays(0)

This is just using a flat curve without any Libor accrual factors.

In [36]:
from financepy.market.curves.FinFlatCurve import FinFlatCurve

In [37]:
discountCurve = FinFlatCurve(valuationDate, 0.05, 2)

Need to specify the next fixing on the floating leg

In [38]:
swap.value(settlementDate, discountCurve, discountCurve)

254422.14362080581

We can examine the individual legs - here is the floating leg valuation

In [39]:
swap.printFloatLeg(valuationDate)

START DATE: WED 20 JUN 2018
MATURITY DATE: SAT 20 SEP 2025
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FLOAT LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
         *** FIRST FLOATING RATE PAYMENT IS IMPLIED ***
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
FRI 20 MAR 2020  0.5055556         0.00   1.00000000         0.00         0.00
MON 21 SEP 2020  0.5138889    253297.64   0.97529598    247040.17    247040.18
MON 22 MAR 2021  0.5055556    249139.38   0.95158817    237078.08    484118.26
MON 20 SEP 2021  0.5055556    249139.38   0.92845666    231315.11    715433.38
MON 21 MAR 2022  0.5055556    249139.38   0.90588744    225692.23    941125.61
TUE 20 SEP 2022  0.5083333    250525.28   0.88374733    221401.05   1162526.66
MON 20 MAR 2023  0.5027778    247753.66   0.86238152    213658.18   1376184.83
WED 20 SEP 2023  0.5111111    251911.37   0.84119096    211905.57   1588090.40
WED 20 MAR 2024  0.5055

And here is the fixed leg valuation

In [40]:
swap.printFixedLeg(valuationDate)

START DATE: WED 20 JUN 2018
MATURITY DATE: SAT 20 SEP 2025
COUPON (%): 5.0
FIXED LEG FREQUENCY: FinFrequencyTypes.ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
MON 21 SEP 2020  1.0194444    509722.22   0.97529598    497130.04    497130.04
MON 20 SEP 2021  1.0111111    505555.56   0.92845666    469386.42    966516.46
TUE 20 SEP 2022  1.0138889    506944.44   0.88374733    448010.80   1414527.26
WED 20 SEP 2023  1.0138889    506944.44   0.84119096    426437.08   1840964.34
FRI 20 SEP 2024  1.0166667    508333.33   0.80057561    406959.27   2247923.61
MON 22 SEP 2025  1.0194444    509722.22   0.76181828    388315.71   2636239.32


In [41]:
swap.pv01(valuationDate, discountCurve)

5.27247864343998

In [42]:
swap.parCoupon(valuationDate, discountCurve)

0.04513544595811375

## Including a final exchange of principal

In [43]:
swap.value(settlementDate, discountCurve, discountCurve, principal=1.0)

254422.14362080768

In [44]:
swap.printFloatLeg(valuationDate)

START DATE: WED 20 JUN 2018
MATURITY DATE: SAT 20 SEP 2025
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FLOAT LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
         *** FIRST FLOATING RATE PAYMENT IS IMPLIED ***
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
FRI 20 MAR 2020  0.5055556         0.00   1.00000000         0.00         0.00
MON 21 SEP 2020  0.5138889    253297.64   0.97529598    247040.17    247040.18
MON 22 MAR 2021  0.5055556    249139.38   0.95158817    237078.08    484118.26
MON 20 SEP 2021  0.5055556    249139.38   0.92845666    231315.11    715433.38
MON 21 MAR 2022  0.5055556    249139.38   0.90588744    225692.23    941125.61
TUE 20 SEP 2022  0.5083333    250525.28   0.88374733    221401.05   1162526.66
MON 20 MAR 2023  0.5027778    247753.66   0.86238152    213658.18   1376184.83
WED 20 SEP 2023  0.5111111    251911.37   0.84119096    211905.57   1588090.40
WED 20 MAR 2024  0.5055

As expected we see the final PV is par.

In [22]:
swap.printFixedLeg(valuationDate)

START DATE: WED 20 JUN 2018
MATURITY DATE: SAT 20 SEP 2025
COUPON (%): 5.0
FIXED LEG FREQUENCY: FinFrequencyTypes.ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE FRI 20 MAR 2020
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
MON 21 SEP 2020  1.0194444    509722.22   0.97529598    497130.04    497130.04
MON 20 SEP 2021  1.0111111    505555.56   0.92845666    469386.42    966516.46
TUE 20 SEP 2022  1.0138889    506944.44   0.88374733    448010.80   1414527.26
WED 20 SEP 2023  1.0138889    506944.44   0.84119096    426437.08   1840964.34
FRI 20 SEP 2024  1.0166667    508333.33   0.80057561    406959.27   2247923.61
MON 22 SEP 2025  1.0194444  10509722.22   0.76181828   8006498.53  10254422.14


Copyright (c) 2020 Dominic O'Kane